In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [5]:
data = pd.read_csv('data.csv')
dataSt = data[['Concentration','MaxWaveLength','MaxPeakAbs','time','Integral','-logc']]
dataSt.dropna(inplace=True)
dataSt

C:\Users\kamya\AppData\Local\Temp\ipykernel_3024\1608507191.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataSt.dropna(inplace=True)


,Concentration,MaxWaveLength,MaxPeakAbs,time,Integral,-logc
0,5,573,0.849,3,371.98,5.301030
1,10,695,0.832,3,425.36,5.000000
2,15,753,0.796,3,421.06,4.823909
3,20,757,0.799,3,417.11,4.698970
4,25,754,0.825,3,425.77,4.602060
...,...,...,...,...,...,...
108,80,754,0.881,3,409.93,4.096910
109,85,746,0.877,3,410.57,4.070581
110,90,749,0.891,3,412.83,4.045757
111,95,748,0.893,3,413.37,4.022276


In [6]:
dataSt['Apeak/A479'] = dataSt['MaxPeakAbs'] / data['479']
dataSt

C:\Users\kamya\AppData\Local\Temp\ipykernel_3024\494904925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataSt['Apeak/A479'] = dataSt['MaxPeakAbs'] / data['479']


,Concentration,MaxWaveLength,MaxPeakAbs,time,Integral,-logc,Apeak/A479
0,5,573,0.849,3,371.98,5.301030,1.250368
1,10,695,0.832,3,425.36,5.000000,1.410169
2,15,753,0.796,3,421.06,4.823909,1.421429
3,20,757,0.799,3,417.11,4.698970,1.450091
4,25,754,0.825,3,425.77,4.602060,1.478495
...,...,...,...,...,...,...,...
108,80,754,0.881,3,409.93,4.096910,1.776210
109,85,746,0.877,3,410.57,4.070581,1.733202
110,90,749,0.891,3,412.83,4.045757,1.778443
111,95,748,0.893,3,413.37,4.022276,1.764822


In [ ]:
sns.pairplot(data[['Concentration','time','Integral','MaxWaveLength']], diag_kind="kde")

In [ ]:
X = dataSt.iloc[:,[1,3,4,6]]
y = dataSt.iloc[:,5]
X

In [ ]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
X = std.fit_transform(X)
y = std.fit_transform(np.array(y).reshape(113,1))
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
from keras import models,layers
model = models.Sequential()
model.add(layers.Dense(16,input_shape=(4,),activation='relu',kernel_initializer='normal'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')


In [ ]:
model.fit(X_train,y_train,epochs=64,batch_size=16)

In [ ]:
from sklearn.metrics import r2_score
pred_train = model.predict(X_train)
print(r2_score(y_train,pred_train))

pred = model.predict(X_test)
print(r2_score(y_test,pred))

KFold Cross validation

In [ ]:
def baseline_model():
    model = models.Sequential()
    model.add(layers.Dense(16,input_shape=(4,),activation='relu',kernel_initializer='normal'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='Adam')
    return model

Evaluate Model


In [ ]:
from keras import models,layers
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.model_selection import KFold

estimator = KerasRegressor(model=baseline_model, epochs=64, batch_size=16, verbose=0)
'''kfold = KFold(n_splits=3,shuffle=True)
results = cross_validate(estimator, X, y, cv=kfold, scoring=('r2', 'neg_mean_squared_error'),return_train_score=True)
print(results['test_neg_mean_squared_error'])
print("train_r2_score:",results['train_r2'])
print("test_r2_score:",results['test_r2'])'''
for k in range(2,10):
    kfold = KFold(n_splits=k,shuffle=True,random_state = 10)
    results = cross_validate(estimator, X, y, cv=kfold, scoring=('r2', 'neg_mean_squared_error'),return_train_score=True)
    print("k = " + str(k))
    print(results['test_neg_mean_squared_error'])
    print("train_r2_score:",results['train_r2'])
    print("test_r2_score:",results['test_r2'])

In [ ]:
estimator.fit(X_train,y_train)


In [ ]:
pred = estimator.predict(X_test)
print(r2_score(y_test,pred))

LinearRegressionModel

In [22]:
data = pd.read_csv('data.csv')
dataSt = data[['MaxWaveLength','MaxPeakAbs','time','Integral','-logc']]
dataSt.dropna(inplace=True)
dataSt['Apeak/A479'] = dataSt['MaxPeakAbs'] / data['479']
dataSt

C:\Users\kamya\AppData\Local\Temp\ipykernel_3024\68290027.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataSt.dropna(inplace=True)
C:\Users\kamya\AppData\Local\Temp\ipykernel_3024\68290027.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataSt['Apeak/A479'] = dataSt['MaxPeakAbs'] / data['479']


,MaxWaveLength,MaxPeakAbs,time,Integral,-logc,Apeak/A479
0,573,0.849,3,371.98,5.301030,1.250368
1,695,0.832,3,425.36,5.000000,1.410169
2,753,0.796,3,421.06,4.823909,1.421429
3,757,0.799,3,417.11,4.698970,1.450091
4,754,0.825,3,425.77,4.602060,1.478495
...,...,...,...,...,...,...
108,754,0.881,3,409.93,4.096910,1.776210
109,746,0.877,3,410.57,4.070581,1.733202
110,749,0.891,3,412.83,4.045757,1.778443
111,748,0.893,3,413.37,4.022276,1.764822


In [23]:
X = dataSt.iloc[:95,[0,2,3,5]]
y = dataSt.iloc[:95,4]
X

,MaxWaveLength,time,Integral,Apeak/A479
0,573,3,371.98,1.250368
1,695,3,425.36,1.410169
2,753,3,421.06,1.421429
3,757,3,417.11,1.450091
4,754,3,425.77,1.478495
...,...,...,...,...
90,769,3,441.06,1.840156
91,765,3,435.63,1.884462
92,768,3,422.23,1.888660
93,766,3,426.51,1.917864


In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(np.array(y).reshape((113,1)))

ValueError: cannot reshape array of size 95 into shape (113,1)

In [25]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X,y)
print("intercept = ",lr.intercept_," coef = ",lr.coef_)
X2 = sm.add_constant(X)
mod = sm.OLS(y,X2)
fii = mod.fit()
print(fii.summary())

intercept =  4.351036502610527  coef =  [ 0.0519729  -0.08920415 -0.11333626 -0.25923097]
                            OLS Regression Results                            
Dep. Variable:                  -logc   R-squared:                       0.917
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     249.7
Date:                Sun, 23 Oct 2022   Prob (F-statistic):           8.05e-48
Time:                        09:49:29   Log-Likelihood:                 94.178
No. Observations:                  95   AIC:                            -178.4
Df Residuals:                      90   BIC:                            -165.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [20]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X,y)
print("intercept = ",lr.intercept_," coef = ",lr.coef_)
X2 = sm.add_constant(X)
mod = sm.OLS(y,X)
fii = mod.fit()
print(fii.summary())

intercept =  4.351036502610527  coef =  [ 0.0519729  -0.08920415 -0.11333626 -0.25923097]
                                 OLS Regression Results                                
Dep. Variable:                  -logc   R-squared (uncentered):                   0.005
Model:                            OLS   Adj. R-squared (uncentered):             -0.039
Method:                 Least Squares   F-statistic:                             0.1075
Date:                Sun, 23 Oct 2022   Prob (F-statistic):                       0.980
Time:                        09:49:19   Log-Likelihood:                         -274.51
No. Observations:                  95   AIC:                                      557.0
Df Residuals:                      91   BIC:                                      567.2
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                 coef    std e

In [21]:
from sklearn.metrics import r2_score
y_pred = lr.predict(X)
r2 = r2_score(y,y_pred)
print(r2)
print(y_test)

0.9173343542901544


NameError: name 'y_test' is not defined